In [ ]:
import altair
import pandas as pd

df = pd.read_csv("datasets/diabetes.csv")

In [ ]:
df.head()

,age,sex,bmi,bp,tc,ldl,hdl,tch,ltg,glu,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [ ]:
form_data = {
    'age': 0.5,      # age in years
    'sex': 0.5,       # sex (1: male, 0: female)
    'bmi': 0.5,    # Body mass index
    'bp':  0.5,      # Average blood pressure
    'tc':  0.5,     # Total serum cholesterol
    'ldl': 0.5,      # Low-density lipoproteins
    'hdl': 0.5,      # High-density lipoproteins
    'tch': 0.5,      # Total cholesterol / HDL
    'ltg': 0.5,     # Log of serum triglycerides level
    'glu': 0.5      # Blood sugar level
}

In [ ]:
df_input = pd.DataFrame([form_data])

# 3) Columns you want to offer in the dropdown
cols = ['age','sex','bmi','bp','tc','ldl','hdl','tch','ltg','glu']

# 4) Define an Altair parameter bound to a dropdown
select_col = alt.param(
    name='Column',  # internal name for the parameter
    bind=alt.binding_select(options=cols, name='Choose X-axis:'),  # label & options
    value='age'  # default selection
)

# 5) Function to create a layer for a given column
def layer_for_col(col):
    """
    Returns a layered chart:
      - Scatter plot of main DataFrame vs. 'target'
      - Regression line
      - Single input data point
      - All hidden by default unless the parameter matches `col`.
    """
    # Main data layer: Add a synthetic field "xcol" for filtering
    base = alt.Chart(df.assign(xcol=col)).transform_filter(
        alt.datum.xcol == select_col  # show this layer only if param == col
    )
    scatter = base.mark_point(color='blue').encode(
        x=alt.X(col),
        y=alt.Y('target')
    )
    reg_line = base.transform_regression(col, 'target').mark_line(color='green')
    
    # Single input data point (same param-based filter)
    input_base = alt.Chart(df_input.assign(xcol=col)).transform_filter(
        alt.datum.xcol == select_col
    )
    input_point = input_base.mark_point(color='red', size=100).encode(
        x=alt.X(col),
        y=alt.Y('target')
    )
    
    return alt.layer(scatter, reg_line, input_point)

# 6) Build one layer per column, then combine & add the param
chart = alt.layer(*(layer_for_col(c) for c in cols)).add_params(select_col)

# 7) Display the chart (e.g., in a Jupyter Notebook)
chart

